## Práctica: implementación del mecanismo de auto-atención con enmascaramiento del modelo Transformer

Vamos a implementar el mecanismo de auto-atención con enmascaramiento del modelo Transformer en Pytorch. Para ello, vamos a seguir los pasos descritos anteriormente y suponer que ya tenemos las matrices de consultas (Q), claves (K) y valores (V) para cada token en la secuencia.

In [15]:
import torch
import math

### Paso 1: Cálculo de las puntuaciones de atención
Matrices de consultas (Q), claves (K) y valores (V):

In [3]:
Q = torch.tensor([[0.0, 0.0, 0.0], [1, 1, 1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3]])
K = torch.tensor([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4]])
V = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [0., 1., 1.]])

El enmascaramiento durante la etapa del decodificador en los modelos Transformer es crucial para evitar que el decodificador tenga acceso a información futura, especialmente en tareas de generación secuencial como la traducción automática o la generación de texto. Este concepto se conoce como "enmascaramiento de atención causal".

En el contexto de los Transformers, el decodificador genera una salida secuencialmente, palabra por palabra. Durante la generación de cada palabra, es importante que el modelo solo tenga en cuenta las palabras anteriores y no las futuras, ya que estas últimas no deberían estar disponibles (en un escenario de generación de texto, por ejemplo, las palabras futuras aún no se han generado).

Una vez realizado el resultado debe ser:

<table>
<tr>
<td><b>z1</b></td><td>1.0000</td><td>0.0000</td><td>0.0000</td>
</tr>
<tr>
<td><b>z2</b></td><td>0.4568</td><td>0.5432</td><td>0.0000</td>
</tr>
<tr>
<td><b>z3</b></td><td>0.3219</td><td>0.3332</td><td>0.3449</td>
</tr>
<tr>
<td><b>z4</b></td><td>0.2309</td><td>0.5130</td><td>0.5260</td>
</tr>
</table>

#### **Objetivos de la práctica**

- Entender con detalle el funcionamiento del mecanismo de auto-atención con enmascaramiento.
- Practicar las operaciones matriciales en PyTorch.

### Paso 2: Escalado de las puntuaciones de atención


In [57]:
Kt = K.t()
QKt = torch.matmul(Q, Kt)
print(QKt)

tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.3000, 0.6000, 0.9000, 1.2000],
        [0.0600, 0.1200, 0.1800, 0.2400],
        [0.0900, 0.1800, 0.2700, 0.3600]])


###  Paso 3: Aplicación de la matriz de enmascaramiento

In [50]:
masked_matrix = torch.tril(QKt)
print(masked_matrix)

tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.3000, 0.6000, 0.0000, 0.0000],
        [0.0600, 0.1200, 0.1800, 0.0000],
        [0.0900, 0.1800, 0.2700, 0.3600]])


### Paso 4: Dividir las puntuaciones de atención por la raíz cuadrada de la dimensión de las consultas

In [63]:
#d_k = Kt.size()
#importar inf
inf = float('-inf')
print(d_k)
scaled_scores = masked_matrix / math.sqrt(3)
scored_inf = scaled_scores.masked_fill(masked_matrix == 0, float('-inf'))
print(scored_inf)

torch.Size([3, 4])
tensor([[  -inf,   -inf,   -inf,   -inf],
        [0.1732, 0.3464,   -inf,   -inf],
        [0.0346, 0.0693, 0.1039,   -inf],
        [0.0520, 0.1039, 0.1559, 0.2078]])


In [61]:
print(scaled_scores)

tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.1732, 0.3464, 0.0000, 0.0000],
        [0.0346, 0.0693, 0.1039, 0.0000],
        [0.0520, 0.1039, 0.1559, 0.2078]])


### Paso 5: Aplicación de la función Softmax

In [62]:
attention = torch.nn.functional.softmax(scaled_scores, dim=1)
print(attention)

tensor([[0.2500, 0.2500, 0.2500, 0.2500],
        [0.2583, 0.3072, 0.2172, 0.2172],
        [0.2455, 0.2542, 0.2631, 0.2372],
        [0.2309, 0.2432, 0.2561, 0.2698]])


In [55]:
print(attention)

tensor([[0.2500, 0.2500, 0.2500, 0.2500],
        [0.2575, 0.2992, 0.2216, 0.2216],
        [0.2461, 0.2536, 0.2614, 0.2389],
        [0.2334, 0.2441, 0.2554, 0.2671]])


### Paso 6: Multiplicación con la matriz de valores (V)

In [56]:
attention_vectors = torch.matmul(attention, V)
print(attention_vectors)

tensor([[0.2500, 0.5000, 0.5000],
        [0.2575, 0.5208, 0.4433],
        [0.2461, 0.4925, 0.5002],
        [0.2334, 0.5112, 0.5225]])


In [41]:
print(Kt)

tensor([[0.1000, 0.2000, 0.3000, 0.4000],
        [0.1000, 0.2000, 0.3000, 0.4000],
        [0.1000, 0.2000, 0.3000, 0.4000]])


In [46]:
from torch.nn.functional import scaled_dot_product_attention
z = scaled_dot_product_attention(Q, K, V, is_causal=True)

In [47]:
print(z)

tensor([[1.0000, 0.0000, 0.0000],
        [0.4568, 0.5432, 0.0000],
        [0.3219, 0.3332, 0.3449],
        [0.2309, 0.5130, 0.5260]])
